In [1]:
import numpy as np
import pandas as pd

df = pd.DataFrame()
df = pd.read_csv('C:/Users/Danny/Documents/fall17/Data Science/train.csv')
df2 = pd.DataFrame()
df2 = pd.read_csv('C:/Users/Danny/Documents/fall17/Data Science/test.csv')

df2.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N


In [50]:
import datetime as datetime
#changes format to dt(datetime) rather than string
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df2['pickup_datetime'] = pd.to_datetime(df2['pickup_datetime'])
#change datetime column to new columns dayofweek, hour, and minute of day
#new columns have numerical values rather than a string 'datetime',
#which Linear Regression can't process
df['day_of_week'] = df['pickup_datetime'].dt.weekday_name
df['hour'] = df['pickup_datetime'].dt.hour
df2['day_of_week'] = df2['pickup_datetime'].dt.weekday_name
df2['hour'] = df2['pickup_datetime'].dt.hour
#minute of day is 60*hour of day + minutes, works well with regression
df['minute_of_day'] = (df['pickup_datetime'].dt.hour*60)+df['pickup_datetime'].dt.minute
df2['minute_of_day'] = (df2['pickup_datetime'].dt.hour*60)+df2['pickup_datetime'].dt.minute
#month as a value of 0-11 
df['month'] = df['pickup_datetime'].dt.month
df2['month'] = df2['pickup_datetime'].dt.month
import calendar
#Assign names to month so OneHotEncoding doesn't overlap integer values
df['month'] = df['month'].apply(lambda x: calendar.month_abbr[x])
df2['month'] = df2['month'].apply(lambda x: calendar.month_abbr[x])
#day of year as numerical (0-364) to account for New Years and other busy days
df['day_of_year'] = df['pickup_datetime'].dt.dayofyear
df2['day_of_year'] = df2['pickup_datetime'].dt.dayofyear
#calculate haversine distance between two (long,lat) pairs
from sklearn.neighbors import DistanceMetric
dist = DistanceMetric.get_metric('haversine')
l = []
for i in range(0,len(df)):
    X = [[df['pickup_latitude'][i],df['pickup_longitude'][i]],
         [df['dropoff_latitude'][i],df['dropoff_longitude'][i]]]
    l.append(dist.pairwise(X)[0][1])
r = []
for i in range(0,len(df2)):
    X = [[df2['pickup_latitude'][i],df2['pickup_longitude'][i]],
         [df2['dropoff_latitude'][i],df2['dropoff_longitude'][i]]]
    r.append(dist.pairwise(X)[0][1])
#create new dataframe to add calculated array to
df3 = pd.DataFrame()
df3['distance'] = l
df = df.join(df3['distance'])
df4 = pd.DataFrame()
df4['distance'] = r
df2 = df2.join(df4['distance'])
#select features that don't need oneHotEncoding
X = df[['pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude','passenger_count', 'minute_of_day','day_of_year','distance']]
X2 = df2[['pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude','passenger_count', 'minute_of_day','day_of_year','distance']]
#normalize
normalized_X=(X - X.mean())/X.std()
normalized_X2=(X2 - X2.mean())/X2.std()
#normalized_X= (X)/X.max()
#day of week and month get one hot encoded because they do not hold comparablevalues in numerical order
oh_dw =pd.get_dummies(df['day_of_week'])
oh_m =pd.get_dummies(df['month'])
oh_dw2 =pd.get_dummies(df2['day_of_week'])
oh_m2 =pd.get_dummies(df2['month'])
#add onehotencoded columns to normalized feature set
normalized_X = normalized_X.join(oh_dw)
normalized_X2 = normalized_X2.join(oh_dw2)
normalized_X = normalized_X.join(oh_m)
normalized_X2 = normalized_X2.join(oh_m2)
normalized_X.fillna(0, inplace=True)
normalized_X2.fillna(0, inplace=True)
#normalized_X2[np.isnan(normalized_X2)] = np.median(normalized_X2[~np.isnan(normalized_X2)])
#normalized_X2

In [51]:
normalized_X.fillna(0, inplace=True)
normalized_X2.fillna(0, inplace=True)
from sklearn.linear_model import LinearRegression
import math
my_linreg = LinearRegression()
y = df['trip_duration']
my_linreg.fit(normalized_X, y)
y_prediction = my_linreg.predict(normalized_X2)

In [52]:
prediction = pd.DataFrame()
prediction['trip_duration'] = y_prediction
prediction[prediction<0]= prediction['trip_duration'].median()
prediction['id'] = df2['id']
prediction.to_csv('C:/Users/Danny/Documents/fall17/Data Science/prediction.csv', encoding='utf-8', index=False)